In [4]:
import numpy as np
import sklearn as sk
import pandas as pd
import numpy.random as rd
import matplotlib.pyplot as plt

%matplotlib inline

from PIL import Image
from PIL import ImageFont, ImageDraw
from pathlib import Path

from IPython.core.debugger import set_trace
import string
import os

pd.options.display.max_rows = 5

In [58]:
# Cicla nos ficheiros das fontes e criar uma dataframe com todas elas.
# A dataframe depois é usada para criar os datasets para o pytorch

# A correr só se for preciso criar de novo as imagens!

paths = [os.path.basename(os.path.normpath(pth)) for pth in Path.cwd().iterdir()]
k = 0

# Tinha feito com estes valores iguais a 50 e, em vez de correr outra vez esta célula, corrigi em baixo
x_image = 32  #50
y_image = 32  #50

k = 0

#for file in Path("./fonts database/all0/").iterdir():
# Por algum motivo não estava a funcionar com o caminho relativo (em cima). Tive que usar o absoluto:

base_dir = "D:/Projects/Thesis/fonts database/"

for file in Path(base_dir + "all0/").iterdir():
    try:
        last_part_path = os.path.basename(os.path.normpath(file))
        file_path = base_dir + "fonts32/" + last_part_path
        file = base_dir + "all0/" + last_part_path
        os.mkdir(file_path[:-4])
        font = ImageFont.truetype(file, 100)
        xmax, ymax = 0, 0

        for character in string.ascii_uppercase:
            x, y = font.getsize(character)
            if x > xmax:
                xmax = x
            if y > ymax:
                ymax = y

        # Tentar ter um tamanho mais uniforme das fontes (ainda há um ou outro caso em que elas saem fora)
        font = ImageFont.truetype(file, int(5000/(xmax+ymax)))


        for character in string.ascii_uppercase:
            x, y = font.getsize(character)
            image = Image.new("L", (x_image,y_image), 255)
    #         image = Image.new("RGBA", (x_image,y_image), (255,255,255))
            draw = ImageDraw.Draw(image)

            draw.fontmode = "1" # (anti)aliasing on

            # Centrar a fonte
    #         draw.text(((x_image - x)/2,(y_image - y)/2), character, (0,0,0), font=font)
            draw.text(((x_image - x)/2,(y_image - y)/2), character, 0, font=font)

            image.save(file_path[:-4] + "/" + font.getname()[0] + " " + font.getname()[1] + " - " + str(character) + ".png")

    except:
        print(file)
        pass

    k += 1
#     if k > 100:
#         break

D:/Projects/Thesis/fonts database/all0/5₧5.ttf
D:/Projects/Thesis/fonts database/all0/aesymatt.ttf
D:/Projects/Thesis/fonts database/all0/desktop.ini
D:/Projects/Thesis/fonts database/all0/FreePixel.ttf
D:/Projects/Thesis/fonts database/all0/Pixel_Berry_08_84_Ltd.Edition.TTF
D:/Projects/Thesis/fonts database/all0/Star-Fox-Starwing.ttf
D:/Projects/Thesis/fonts database/all0/TLOZ-Minish-Cap-A-Link-to-the-Past-Four-Sword.ttf


In [92]:
# Cuidado para não sobrepôr os dados!!
# Eu transformo em df depois de transformar em imagens e não ao mesmo tempo porque
# ainda estive a filtrar algumas das fontes, que estavam fora ou tinham outros problemas.
# Baseio-me portanto nas fontes em fonts50

# Listas para pôr dados a enviar para o pandas
fonts_arrays = []
fonts_names = []

for i, folder in enumerate(Path(base_dir + "fonts32").iterdir()):

    # Dicionário com as colunas a ser enviadas para o pandas
    dict_font = {}
    k = 0
    for character, file in zip(string.ascii_uppercase, Path(folder).iterdir()):
        img = Image.open(file)
        img_array = np.array(img)
        if k == 0 and img_array.shape == (50, 50):
            print(img_array.shape, file, )
        # As np.float32 because pytorch likes flaot32?
        # I made an error before and created 50X50 pics instead of 48X48, so I undo this here
        img_array = np.where(img_array == 255., 0., 1.) #.astype(np.float32)[1:-1, 1:-1]
        dict_font.update({character: img_array})
        k += 1
    fonts_arrays.append(dict_font)

    font_name = os.path.basename(os.path.normpath(file))[:-8]
    fonts_names.append(font_name)

In [93]:
fonts32_df = pd.DataFrame(fonts_arrays)
fonts32_df['font_name'] = fonts_names

In [94]:
fonts32_df.to_pickle(base_dir + 'fonts32_df.pkl')

In [8]:
# Pôr mais bonito, sem loops (Tive que criar lista para ser mais facil criar o dataframe para passar para o dataset)
X = [None]*len(fonts48_df) # Se mudar tamanho mudar aqui
for i in range(len(fonts48_df)):
    X[i] = np.zeros((25, 48, 48), dtype=np.float32)
    for j in range(25):
        X[i][j] = fonts48_df.loc[i][j]

y = [None]*len(y_)
for i in range(len(y_)):
    y[i] = y_[i].astype(np.float32)